In [9]:
# Import Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import pandas as pd

In [2]:
# Connect to database
engine = create_engine("sqlite:///politics_db.db", connect_args={'check_same_thread': False}, echo=True)

# Reflect database and tables into new models
Base = automap_base()

In [3]:
Base.prepare(engine, reflect=True)
Candidates = Base.classes.candidates
Twitter = Base.classes.twitter

# Create our session from Python to the DB
session = Session(bind=engine)

2019-08-11 14:44:58,779 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-08-11 14:44:58,782 INFO sqlalchemy.engine.base.Engine ()
2019-08-11 14:44:58,798 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-08-11 14:44:58,800 INFO sqlalchemy.engine.base.Engine ()
2019-08-11 14:44:58,805 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-08-11 14:44:58,810 INFO sqlalchemy.engine.base.Engine ()
2019-08-11 14:44:58,815 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("candidates")
2019-08-11 14:44:58,820 INFO sqlalchemy.engine.base.Engine ()
2019-08-11 14:44:58,823 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'candidates' AND type = 'table'
2019-08-11 14:44:58,825 INFO sqlalchemy.engine.base.Engine ()
2019-08-11 14:44:58,832 INFO sqlalchemy.en

In [4]:
username = "@JoeBiden"

In [5]:
# Get sentiment data 
results = session.query(Twitter.tweet_date, Twitter.sentiment, func.count(Twitter.id).label("tweet_count"))\
    .group_by(Twitter.tweet_date)\
    .group_by(Twitter.sentiment)\
    .filter(Twitter.username == username)\
    .all()

2019-08-11 14:45:04,435 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-08-11 14:45:04,439 INFO sqlalchemy.engine.base.Engine SELECT twitter.tweet_date AS twitter_tweet_date, twitter.sentiment AS twitter_sentiment, count(twitter.id) AS tweet_count 
FROM twitter 
WHERE twitter.username = ? GROUP BY twitter.tweet_date, twitter.sentiment
2019-08-11 14:45:04,447 INFO sqlalchemy.engine.base.Engine ('@JoeBiden',)


In [13]:
# Convert query result into 2 lists of dictionaries: one for positive, one for negative
positive_tweets = []
negative_tweets = []

for result in results:
    if result.sentiment:
        positive_count = result.tweet_count
        tweet_dict = {"date": result.tweet_date, "positive_count": positive_count}
        positive_tweets.append(tweet_dict)
    else:
        negative_count = result.tweet_count
        tweet_dict = {"date": result.tweet_date, "negative_count": negative_count}
        negative_tweets.append(tweet_dict)

In [14]:
# Convert lists to dataframe and merge on date
positive_df = pd.DataFrame(positive_tweets)
negative_df = pd.DataFrame(negative_tweets)
twitter_df = pd.merge(positive_df, negative_df, on = "date")

# Returned dataframe as json object
jsonified = twitter_df.to_json(orient = "records")

'[{"date":"2019-08-01","positive_count":7903,"negative_count":14273},{"date":"2019-08-02","positive_count":5287,"negative_count":8720},{"date":"2019-08-03","positive_count":4406,"negative_count":7439},{"date":"2019-08-04","positive_count":1353,"negative_count":2310},{"date":"2019-08-05","positive_count":1342,"negative_count":2228},{"date":"2019-08-06","positive_count":1107,"negative_count":1978}]'